In [2]:
! pip install python-pptx

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------  3.9/4.0 MB 23.4 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 18.5 MB/s  0:00:00

   ------------- -------------------------- 1/3 [lxml]
   -------------------------- ------------- 2/3 [python-pptx]
   -------------------------- ------------- 2/3 [python-pptx]
   ---------------------------------------- 3/3 [python-pptx]



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import os

# -----------------------------------------------------------------------------
# 1. 차트 생성 함수 (Dummy Data 활용 - 시각화용)
# -----------------------------------------------------------------------------
def create_charts():
    # 스타일 설정
    plt.style.use('ggplot')
    
    # 1-1. Feature Importance Chart
    features = ['Product_ID_Mean', 'Product_Category', 'Age', 'Occupation', 'Gender']
    importance = [0.92, 0.05, 0.01, 0.01, 0.01]
    
    plt.figure(figsize=(6, 4))
    bars = plt.barh(features[::-1], importance[::-1], color='#4472C4')
    plt.title('Feature Importance (XGBoost)')
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.savefig('ppt_chart_importance.png', dpi=100)
    plt.close()

    # 1-2. Actual vs Predicted Scatter Plot
    np.random.seed(42)
    actual = np.random.normal(10000, 5000, 200)
    predicted = actual + np.random.normal(0, 2000, 200) # Noise added
    
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=actual, y=predicted, alpha=0.6, color='#ED7D31')
    plt.plot([0, 25000], [0, 25000], 'r--', label='Ideal Fit')
    plt.title('Actual vs Predicted Purchase')
    plt.xlabel('Actual ($)')
    plt.ylabel('Predicted ($)')
    plt.legend()
    plt.tight_layout()
    plt.savefig('ppt_chart_scatter.png', dpi=100)
    plt.close()

# 차트 이미지 생성 실행
create_charts()

# -----------------------------------------------------------------------------
# 2. 프레젠테이션 생성 로직
# -----------------------------------------------------------------------------
prs = Presentation()

# 슬라이드 마스터 레이아웃 인덱스 (일반적인 템플릿 기준)
TITLE_SLIDE_LAYOUT = 0
BULLET_SLIDE_LAYOUT = 1
Content_SLIDE_LAYOUT = 5 # Title Only (for custom layouts)

def add_slide(prs, layout_idx, title, content_text=None):
    slide = prs.slides.add_slide(prs.slide_layouts[layout_idx])
    title_placeholder = slide.shapes.title
    title_placeholder.text = title
    
    if content_text and layout_idx == BULLET_SLIDE_LAYOUT:
        tf = slide.shapes.placeholders[1].text_frame
        tf.text = content_text[0] # 첫 줄
        for line in content_text[1:]:
            p = tf.add_paragraph()
            p.text = line
            p.level = 1 if line.startswith('-') else 0
            if line.startswith('-'):
                p.text = line[1:].strip() # 하이픈 제거 후 들여쓰기
                
    return slide

# --- Slide 1: Title ---
slide = prs.slides.add_slide(prs.slide_layouts[TITLE_SLIDE_LAYOUT])
title = slide.shapes.title
subtitle = slide.shapes.placeholders[1]
title.text = "월마트 매출 예측 모델 고도화 보고서"
subtitle.text = "수치해석적 최적화와 Data-Centric 전략\n\n작성자: 데이터 분석팀 (수치해석 전공)\n2026년 2월 11일"

# --- Slide 2: 개요 ---
content = [
    "연구 목적: 월마트 블랙프라이데이 데이터 기반 구매 금액 예측",
    "핵심 과제:",
    "- High Cardinality(Product_ID) 변수의 효과적 처리",
    "- 결측값 및 신규 상품(Cold Start)에 대한 강건성 확보",
    "- 수치해석적 기법(Gradient Boosting)을 통한 최적화"
]
add_slide(prs, BULLET_SLIDE_LAYOUT, "1. 프로젝트 개요", content)

# --- Slide 3: 연구 가설 ---
content = [
    "H1 (인구통계 영향력): 기각",
    "- 사용자의 나이, 성별은 가격 결정에 큰 영향을 주지 않을 것이다.",
    "H2 (상품 가치 지배력): 채택",
    "- 구매 금액은 '무엇을 샀는가(Product Value)'에 의해 결정될 것이다.",
    "H3 (직교성 확보): 채택",
    "- K-Fold Target Encoding은 타겟 노이즈와 직교(Orthogonal)하여 과적합을 막을 것이다."
]
add_slide(prs, BULLET_SLIDE_LAYOUT, "2. 연구 가설 및 검증", content)

# --- Slide 4: 방법론 (Methodology) ---
content = [
    "Feature Engineering: K-Fold Target Encoding",
    "- Data Leakage 방지를 위해 Out-of-Fold 평균 사용",
    "- 수식: x_hat_i = Mean(Target) of {Data - Fold_k}",
    "",
    "Algorithm: XGBoost (eXtreme Gradient Boosting)",
    "- 손실 함수의 2차 미분(Hessian) 활용",
    "- 정규화(L1, L2) 파라미터를 통한 일반화 성능 극대화"
]
add_slide(prs, BULLET_SLIDE_LAYOUT, "3. 모델링 방법론", content)

# --- Slide 5: 실험 결과 (표) ---
# 표가 들어갈 슬라이드 (Title Only)
slide = prs.slides.add_slide(prs.slide_layouts[Content_SLIDE_LAYOUT])
slide.shapes.title.text = "4. 모델 성능 평가 결과"

# 표 데이터 생성
rows, cols = 3, 4
left = Inches(1)
top = Inches(2)
width = Inches(8)
height = Inches(3)

table = slide.shapes.add_table(rows, cols, left, top, width, height).table
# 헤더
table.cell(0, 0).text = "Model"
table.cell(0, 1).text = "RMSE ($)"
table.cell(0, 2).text = "R² Score"
table.cell(0, 3).text = "Note"
# 데이터
table.cell(1, 0).text = "Baseline (RF)"
table.cell(1, 1).text = "2,687"
table.cell(1, 2).text = "0.7126"
table.cell(1, 3).text = "Basic Label Encoding"

table.cell(2, 0).text = "Advanced (XGB)"
table.cell(2, 1).text = "2,670"
table.cell(2, 2).text = "0.7161"
table.cell(2, 3).text = "K-Fold TE + Tuning"

# --- Slide 6: 시각화 (이미지 삽입) ---
slide = prs.slides.add_slide(prs.slide_layouts[Content_SLIDE_LAYOUT])
slide.shapes.title.text = "5. 상세 분석: 시각화"

# 이미지 삽입
img_path1 = 'ppt_chart_importance.png'
img_path2 = 'ppt_chart_scatter.png'

if os.path.exists(img_path1):
    slide.shapes.add_picture(img_path1, Inches(0.5), Inches(2), width=Inches(4.5))
if os.path.exists(img_path2):
    slide.shapes.add_picture(img_path2, Inches(5.2), Inches(2), width=Inches(4.5))

# 캡션 추가
txBox = slide.shapes.add_textbox(Inches(1), Inches(6), Inches(8), Inches(1))
tf = txBox.text_frame
tf.text = "Insight: Product_ID 변수의 영향력이 절대적이며(좌), 예측값은 실제 정가(Ideal Fit)를 따름(우)."

# --- Slide 7: 원인 분석 ---
content = [
    "가격 경직성 (Price Rigidity)",
    "- 마트 상품 가격은 정찰제로 운영됨 (Fixed Price).",
    "- 사용자 특성(나이, 성별)에 따른 가격 변동(Variance)이 거의 없음.",
    "",
    "한계점",
    "- 현재 데이터만으로는 '할인'이나 '프로모션'에 의한 미세 변동 설명 불가.",
    "- 알고리즘 튜닝은 한계 효용에 도달함."
]
add_slide(prs, BULLET_SLIDE_LAYOUT, "6. 원인 분석 (Root Cause)", content)

# --- Slide 8: 결론 및 제언 ---
content = [
    "결론: Robust Baseline 확보",
    "- 신규 상품 유입 시에도 인구통계 정보를 이용한 보간(Interpolation) 가능.",
    "",
    "Data-Centric Strategy 제안 (R² > 0.8 목표)",
    "- 1. 할인율(Discount Rate) 데이터 확보",
    "- 2. 시계열(Time Series) 변수 추가 (월초/월말, 공휴일)",
    "- 3. 유저 프로파일링(User History) 변수 생성"
]
add_slide(prs, BULLET_SLIDE_LAYOUT, "7. 결론 및 향후 전략", content)

# --- Slide 9: Q&A ---
slide = prs.slides.add_slide(prs.slide_layouts[TITLE_SLIDE_LAYOUT])
title = slide.shapes.title
title.text = "Q & A"
slide.shapes.placeholders[1].text = "감사합니다."

# 파일 저장
output_file = "Walmart_Analysis_Report.pptx"
prs.save(output_file)
print(f"Successfully generated {output_file}")

# 임시 이미지 파일 삭제
if os.path.exists(img_path1): os.remove(img_path1)
if os.path.exists(img_path2): os.remove(img_path2)

Successfully generated Walmart_Analysis_Report.pptx
